In [13]:
import os


class Para:
    read_folder_path = 'data/sk_space_1d_rate_20d_12-21_raw'
    write_folder_path = 'data/sk_space_1d_rate_20d_12-21_pre'

    feature_column_start_name = 'ep_ratio_ttm'
    feature_column_end_name = 'BR'

    month_num = len(os.listdir(read_folder_path))
para = Para()
print(para.month_num)

2411


In [14]:
import time
import numpy as np
import pandas as pd


time_begin = time.time()

df = pd.DataFrame()

for i_csv in range(para.month_num):

    df_curr_month = pd.read_csv(para.read_folder_path +'/'+ str(i_csv) + '.csv', index_col='order_book_id')

    df_curr_month['date'] = pd.to_datetime(df_curr_month['date'])

    df = pd.concat([df, df_curr_month])

df

,board_type,sector_code,yield_rate,month,date,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,MACD_DIFF,MACD_DEA,...,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR
order_book_id,,,,,,,,,,,,,,,,,,,,,
600376.XSHG,MainBoard,RealEstate,-0.008650,0,2012-01-04,0.044343,1.311060,0.391262,0.266303,0.187202,...,60.089686,66.666667,5.786825,1.562536,1.664137,1.615586,0.193960,0.234169,197.375328,147.478992
600376.XSHG,MainBoard,RealEstate,0.034391,1,2012-01-05,0.045524,1.277044,0.401684,0.248129,0.199388,...,56.066946,66.666667,2.678063,1.607689,1.682598,1.611548,0.191696,0.237295,190.452261,141.649899
600376.XSHG,MainBoard,RealEstate,0.088488,2,2012-01-06,0.045373,1.281296,0.400351,0.233455,0.206201,...,62.272727,66.666667,2.744786,1.597929,1.662417,1.608140,0.192225,0.240303,175.462963,130.841121
600376.XSHG,MainBoard,RealEstate,0.013847,3,2012-01-09,0.043728,1.329486,0.385839,0.246421,0.214245,...,64.069264,66.666667,6.090595,1.624533,1.657415,1.607297,0.198988,0.246421,172.185430,124.210526
600376.XSHG,MainBoard,RealEstate,-0.016509,4,2012-01-10,0.042373,1.372007,0.373882,0.277703,0.226937,...,62.272727,66.666667,8.751826,1.678813,1.662655,1.618777,0.213578,0.258807,183.783784,132.263815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600376.XSHG,MainBoard,RealEstate,0.166316,2406,2021-11-29,0.201045,0.388018,5.202812,-0.070733,-0.094523,...,48.387097,33.333333,0.486567,0.365571,0.311826,0.242845,0.051514,0.295456,72.588832,53.138075
600376.XSHG,MainBoard,RealEstate,0.186441,2407,2021-11-30,0.202323,0.385567,5.235881,-0.068587,-0.089336,...,46.875000,33.333333,-0.190315,0.354401,0.311464,0.242669,0.050357,0.298859,89.221557,69.473684
600376.XSHG,MainBoard,RealEstate,0.173640,2408,2021-12-01,0.199783,0.390469,5.170159,-0.061337,-0.083736,...,58.333333,44.444444,1.067766,0.351969,0.311105,0.243638,0.050833,0.301140,102.666667,81.286550


In [15]:
# Replace infinite updated data with nan
df.replace(to_replace=[np.inf, -np.inf], value=np.nan, inplace=True)

# 剔除含有 na 的行
df_dropna = df.dropna(axis=0, how='any')

# 选择特征列和标签列
df_dropna_y = df_dropna[['board_type', 'sector_code', 'month', 'date']]
df_dropna_features = df_dropna.loc[:, para.feature_column_start_name:para.feature_column_end_name]
df_dropna_features = pd.concat([df_dropna['yield_rate'], df_dropna_features], axis=1)

# 中位数去极值
df_dropna_features = df_dropna_features.clip(
    lower=df_dropna_features.median(axis=0) - 5 * df_dropna_features.mad(axis=0),
    upper=df_dropna_features.median(axis=0) + 5 * df_dropna_features.mad(axis=0),
    axis=1)

# 标准化
df_dropna_features = (df_dropna_features - df_dropna_features.mean()) / df_dropna_features.std(ddof=0)

# 拼接标签列和特征列
df_dropna_standard = pd.concat([df_dropna_y, df_dropna_features], axis=1)

# 写入文件
df_dropna_standard.to_csv(para.write_folder_path + '/0.csv')

print()
print('Done!')


Done!
